In [2]:
# Import Libraries here
from tensorflow.keras import datasets, layers, models
import numpy as np
import matplotlib.pylab as plt
import os
from datetime import datetime
import tensorflow as tf

In [4]:
# load and normalize images
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

# class labels
CLASS_NAMES = ['airplane', 'automobile', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck']

170508288/170498071 [==============================] - 381s 2us/step


In [5]:
# 500 images for Validation
validation_dataset = tf.data.Dataset.from_tensor_slices((test_images[:500], test_labels[:500]))

# the rest for Testing
test_dataset = tf.data.Dataset.from_tensor_slices((test_images[500:], test_labels[500:]))

# prep train Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))

# sample size of each dataset
train_dataset_size = len(list(train_dataset.as_numpy_iterator()))
validation_dataset_size = len(list(validation_dataset.as_numpy_iterator()))
test_dataset_size = len(list(test_dataset.as_numpy_iterator()))

print('Training data sample size: ', train_dataset_size)
print('Validation data sample size: ', validation_dataset_size)
print('Test data sample size: ', test_dataset_size)

Training data sample size:  50000
Validation data sample size:  500
Test data sample size:  9500


In [6]:
# define a Distribution strategy for training
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync)) # No GPU devices

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of devices: 1


In [7]:
# set BATCH SIZE for training
BUFFER_SIZE = 10000
BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

train_dataset = train_dataset.repeat().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
validation_dataset = validation_dataset.shuffle(BUFFER_SIZE).batch(validation_dataset_size)
test_dataset = test_dataset.batch(test_dataset_size)

STEPS_PER_EPOCH = train_dataset_size // BATCH_SIZE_PER_REPLICA # 781
VALIDATION_STEPS = 1

In [8]:
# define the function to BUILD the model
def build_model():
    with strategy.scope():
        model = tf.keras.Sequential([
            layers.Conv2D(
                32, kernel_size=(3,3), activation='relu', name='conv_1',
                kernel_initializer='glorot_uniform', padding='same', input_shape=(32, 32, 3)
            ),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(
                64, kernel_size=(3, 3), activation='relu', name='conv_2',
                kernel_initializer='glorot_uniform', padding='same'
            ),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(name='flat_1'),
            layers.Dense(256, activation='relu', kernel_initializer='glorot_uniform', name='dense_64'),
            layers.Dense(10, activation='softmax', name='custom_class')
        ])
        model.build([None, 32, 32, 3])
        model.compile(
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy']
        )
        return model

# invoke the model
model = build_model()
MODEL_NAME = 'myCIFAR10-{}'.format(datetime.now().strftime("%Y%m%d-%H%M%S"))
print(MODEL_NAME)

checkpoint_dir = './' + MODEL_NAME
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt-{epoch}")

myCheckPoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, monitor='val_accuracy', mode='max',
    save_weights_only=True, save_best_only=True
)

myCallbacks = [myCheckPoint]

myCIFAR10-20220120-143834


In [10]:
# train the model
model.fit(
    train_dataset, epochs=30, steps_per_epoch=STEPS_PER_EPOCH, validation_data=validation_dataset,
    validation_steps=VALIDATION_STEPS, callbacks=myCallbacks
)

Epoch 1/30


C:\Users\DELL\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


781/781 [==============================] - 64s 77ms/step - loss: 1.3748 - accuracy: 0.5162 - val_loss: 1.1050 - val_accuracy: 0.5960
Epoch 2/30
781/781 [==============================] - 61s 79ms/step - loss: 1.0129 - accuracy: 0.6477 - val_loss: 0.9867 - val_accuracy: 0.6540
Epoch 3/30
781/781 [==============================] - 63s 81ms/step - loss: 0.8753 - accuracy: 0.6944 - val_loss: 0.9563 - val_accuracy: 0.6800
Epoch 4/30
781/781 [==============================] - 62s 79ms/step - loss: 0.7631 - accuracy: 0.7337 - val_loss: 0.8867 - val_accuracy: 0.6780
Epoch 5/30
781/781 [==============================] - 62s 80ms/step - loss: 0.6605 - accuracy: 0.7710 - val_loss: 0.8766 - val_accuracy: 0.7060
Epoch 6/30
781/781 [==============================] - 71s 91ms/step - loss: 0.5688 - accuracy: 0.8014 - val_loss: 0.9317 - val_accuracy: 0.6940
Epoch 7/30
781/781 [==============================] - 71s 91ms/step - loss: 0.4799 - accuracy: 0.8338 - val_loss: 1.0433 - val_accuracy: 0.6600
Epo

In [14]:
# to load the best weights to the model
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

# check for latest model checkpoint in the DIR
tf.train.latest_checkpoint(checkpoint_dir)

'./myCIFAR10-20220120-143834\\ckpt-9'

In [18]:
# save MODEL to h5 format
KERAS_MODEL_PATH = r'C:\Users\DELL\Desktop\Learning projects\assests\tfkeras_cifar10.h5'
model.save(KERAS_MODEL_PATH)

# to reload model
new_h5_model = models.load_model(KERAS_MODEL_PATH)


In [19]:
# 9500 rows
new_h5_model.predict(test_dataset)


array([[9.01132735e-05, 2.75433422e-07, 4.76483814e-02, ...,
        4.20681387e-03, 3.71989408e-08, 6.65380696e-07],
       [3.07483482e-04, 5.94857283e-07, 9.25968122e-03, ...,
        2.06165605e-05, 2.26582819e-09, 4.09300718e-08],
       [2.07293968e-04, 4.43747723e-08, 4.47228970e-03, ...,
        1.50847882e-01, 5.35844767e-04, 2.81508278e-06],
       ...,
       [6.41857014e-06, 1.19893956e-11, 1.20093515e-02, ...,
        1.95345515e-03, 9.74205591e-07, 3.95296702e-08],
       [1.71055943e-01, 4.78979856e-01, 5.90977678e-03, ...,
        2.53810049e-06, 6.19740661e-07, 6.93959095e-08],
       [3.58529895e-09, 3.47679929e-06, 6.87571128e-06, ...,
        9.99704897e-01, 9.30349664e-09, 2.19125639e-07]], dtype=float32)

Protobuf Model

In [21]:
# to save model to PB format, specifically use this
SAVED_MODEL_PATH = r'C:\Users\DELL\Desktop\Learning projects\assests\pb_model'
tf.saved_model.save(model, SAVED_MODEL_PATH)


INFO:tensorflow:Assets written to: C:\Users\DELL\Desktop\Learning projects\assests\pb_model\assets


In [ ]:
# load a Protobuf model
# this would work better with a GPU 
load_strategy = tf.distribute.MirroredStrategy()
with load_strategy.scope():
    load_options = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
    loaded_pb = models.load_model(SAVED_MODEL_PATH, options=load_options)


# to predict with PB model
loaded_pb.predict(test_dataset)


The model was served using Docker...



In [ ]:
# client code which should be in a diff Notebook

import tensorflow as tf
from tensorflow.keras import datasets
import requests
import json
import numpy as np

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0
test_images = test_images[500:510]

DATA = json.dumps({"instances": test_images.tolist()})
HEADERS = {'content-type': 'application/json'}

# TFS would score the DATA and return the results as response
response = requests.post('http://localhost:8501/v1/models/cifar10:predict', data=DATA, headers=HEADERS)

# to see raw response
predictions_prob_list = response.json().get('predictions')
CLASS_NAMES = ['airplane', 'automobile', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck']


predictions_array = np.asarray(predictions_prob_list)
predictions_idx = np.argmax(predictions_array, axis=0)


